In [2]:
import usdaquickstats
u = usdaquickstats.USDAQuickStats()
u.getparams["year"] = 2012
u.getparams["state_alpha"] = ("MO","TX")
u.getparams["commodity_desc"] = "CORN"
u.get()

Got 10222 records.


In [3]:
import pandas as pd
import io
pd.set_option('display.max_row',100) #set the default number of rows to display when printing the df
pd.set_option('display.max_columns',50) #set the default max number of columns to display when printing the df
df = pd.read_csv(io.StringIO(u.response.content.decode('utf-8')))
df.head(5)

,source_desc,sector_desc,group_desc,commodity_desc,class_desc,prodn_practice_desc,util_practice_desc,statisticcat_desc,unit_desc,short_desc,domain_desc,domaincat_desc,agg_level_desc,state_ansi,state_fips_code,state_alpha,state_name,asd_code,asd_desc,county_ansi,county_code,county_name,region_desc,zip_5,watershed_code,watershed_desc,congr_district_code,country_code,country_name,location_desc,year,freq_desc,begin_code,end_code,reference_period_desc,week_ending,load_time,Value,CV (%)
0,CENSUS,CROPS,FIELD CROPS,CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,SALES,$,"CORN - SALES, MEASURED IN $",AREA OPERATED,"AREA OPERATED: (1,000 TO 1,999 ACRES)",STATE,29,29,MO,MISSOURI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,9000,UNITED STATES,MISSOURI,2012,ANNUAL,0,0,YEAR,NaN,2012-12-31 00:00:00,"401,093,000",1.5
1,CENSUS,CROPS,FIELD CROPS,CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,SALES,$,"CORN - SALES, MEASURED IN $",AREA OPERATED,"AREA OPERATED: (1,000 TO 1,999 ACRES)",STATE,48,48,TX,TEXAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,9000,UNITED STATES,TEXAS,2012,ANNUAL,0,0,YEAR,NaN,2012-12-31 00:00:00,"291,081,000",8.7
2,CENSUS,CROPS,FIELD CROPS,CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,SALES,$,"CORN - SALES, MEASURED IN $",AREA OPERATED,AREA OPERATED: (1.0 TO 9.9 ACRES),STATE,29,29,MO,MISSOURI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,9000,UNITED STATES,MISSOURI,2012,ANNUAL,0,0,YEAR,NaN,2012-12-31 00:00:00,"85,000",20.6
3,CENSUS,CROPS,FIELD CROPS,CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,SALES,$,"CORN - SALES, MEASURED IN $",AREA OPERATED,AREA OPERATED: (1.0 TO 9.9 ACRES),STATE,48,48,TX,TEXAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,9000,UNITED STATES,TEXAS,2012,ANNUAL,0,0,YEAR,NaN,2012-12-31 00:00:00,(D),(D)
4,CENSUS,CROPS,FIELD CROPS,CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,SALES,$,"CORN - SALES, MEASURED IN $",AREA OPERATED,AREA OPERATED: (10.0 TO 49.9 ACRES),STATE,29,29,MO,MISSOURI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,9000,UNITED STATES,MISSOURI,2012,ANNUAL,0,0,YEAR,NaN,2012-12-31 00:00:00,"3,853,000",8.6


Next we'll try summarizing some data in a pandas dataframe

http://bconnelly.net/2013/10/summarizing-data-in-python-with-pandas/

Let's first take a look at the columns and data types in the data frame.

In [4]:
df.dtypes

source_desc               object
sector_desc               object
group_desc                object
commodity_desc            object
class_desc                object
prodn_practice_desc       object
util_practice_desc        object
statisticcat_desc         object
unit_desc                 object
short_desc                object
domain_desc               object
domaincat_desc            object
agg_level_desc            object
state_ansi                 int64
state_fips_code            int64
state_alpha               object
state_name                object
asd_code                 float64
asd_desc                  object
county_ansi              float64
county_code              float64
county_name               object
region_desc              float64
zip_5                    float64
watershed_code             int64
watershed_desc           float64
congr_district_code      float64
country_code               int64
country_name              object
location_desc             object
year      

Now we can create a groupbyobject and produce some summaries for those groups.

First we need to create a numeric version of the Value field since the original field contains string values for the suppressed cells i.e. (D). Because the original column is an 'object' (string) data type, values that included commas for the million and thousand delimiters are stored as, for example, "1,000" instead of "1000". 

So we use coerce to convert values to numbers when possible.

In [5]:
import numpy as np

df['ValueNum'] = pd.to_numeric(df['Value'].str.replace(",",""),errors='coerce')

groups = df[df.short_desc.str.contains('OPERATIONS')].groupby(['state_alpha','short_desc'])

pd.set_option('display.float_format',lambda x: '%.3f' % x)

dfg = groups['ValueNum'].aggregate([np.mean,np.sum,np.count_nonzero])

dfg

mean  \
state_alpha short_desc                                                     
MO          CORN - OPERATIONS WITH SALES                        1004.358   
            CORN - SALES, MEASURED IN PCT OF FARM OPERATIONS      15.700   
            CORN, GRAIN - OPERATIONS WITH AREA HARVESTED         345.475   
            CORN, GRAIN, IRRIGATED - OPERATIONS WITH AREA H...    94.229   
            CORN, GRAIN, IRRIGATED, ENTIRE CROP - OPERATION...   466.000   
            CORN, GRAIN, IRRIGATED, NONE OF CROP - OPERATIO... 13945.000   
            CORN, GRAIN, IRRIGATED, PART OF CROP - OPERATIO...   690.000   
            CORN, SILAGE - OPERATIONS WITH AREA HARVESTED         79.304   
            CORN, SILAGE, IRRIGATED - OPERATIONS WITH AREA ...     6.961   
            CORN, SILAGE, IRRIGATED, ENTIRE CROP - OPERATIO...    20.000   
            CORN, SILAGE, IRRIGATED, NONE OF CROP - OPERATI...  2435.000   
            CORN, SILAGE, IRRIGATED, PART OF CROP - OPERATI...    30.000   
TX          CORN - OPERATIONS WITH SALES                         292.775   
            CORN - SALES, MEASURED IN PCT OF FARM OPERATIONS       1.700   
            CORN, GRAIN - OPERATIONS WITH AREA HARVESTED          89.183   
            CORN, GRAIN, IRRIGATED - OPERATIONS WITH AREA H...   131.134   
            CORN, GRAIN, IRRIGATED, ENTIRE CROP - OPERATION...  1535.000   
            CORN, GRAIN, IRRIGATED, NONE OF CROP - OPERATIO...  2249.000   
            CORN, GRAIN, IRRIGATED, PART OF CROP - OPERATIO...   261.000   
            CORN, SILAGE - OPERATIONS WITH AREA HARVESTED         25.111   
            CORN, SILAGE, IRRIGATED - OPERATIONS WITH AREA ...    45.732   
            CORN, SILAGE, IRRIGATED, ENTIRE CROP - OPERATIO...   373.000   
            CORN, SILAGE, IRRIGATED, NONE OF CROP - OPERATI...   226.000   
            CORN, SILAGE, IRRIGATED, PART OF CROP - OPERATI...    23.000   

                                                                      sum  \
state_alpha short_desc                                                      
MO          CORN - OPERATIONS WITH SALES                       272181.000   
            CORN - SALES, MEASURED IN PCT OF FARM OPERATIONS       15.700   
            CORN, GRAIN - OPERATIONS WITH AREA HARVESTED       427007.000   
            CORN, GRAIN, IRRIGATED - OPERATIONS WITH AREA H...  16867.000   
            CORN, GRAIN, IRRIGATED, ENTIRE CROP - OPERATION...    466.000   
            CORN, GRAIN, IRRIGATED, NONE OF CROP - OPERATIO...  13945.000   
            CORN, GRAIN, IRRIGATED, PART OF CROP - OPERATIO...    690.000   
            CORN, SILAGE - OPERATIONS WITH AREA HARVESTED       68281.000   
            CORN, SILAGE, IRRIGATED - OPERATIONS WITH AREA ...    717.000   
            CORN, SILAGE, IRRIGATED, ENTIRE CROP - OPERATIO...     20.000   
            CORN, SILAGE, IRRIGATED, NONE OF CROP - OPERATI...   2435.000   
            CORN, SILAGE, IRRIGATED, PART OF CROP - OPERATI...     30.000   
TX          CORN - OPERATIONS WITH SALES                        96323.000   
            CORN - SALES, MEASURED IN PCT OF FARM OPERATIONS        1.700   
            CORN, GRAIN - OPERATIONS WITH AREA HARVESTED       118168.000   
            CORN, GRAIN, IRRIGATED - OPERATIONS WITH AREA H...  26489.000   
            CORN, GRAIN, IRRIGATED, ENTIRE CROP - OPERATION...   1535.000   
            CORN, GRAIN, IRRIGATED, NONE OF CROP - OPERATIO...   2249.000   
            CORN, GRAIN, IRRIGATED, PART OF CROP - OPERATIO...    261.000   
            CORN, SILAGE - OPERATIONS WITH AREA HARVESTED       17352.000   
            CORN, SILAGE, IRRIGATED - OPERATIONS WITH AREA ...   5808.000   
            CORN, SILAGE, IRRIGATED, ENTIRE CROP - OPERATIO...    373.000   
            CORN, SILAGE, IRRIGATED, NONE OF CROP - OPERATI...    226.000   
            CORN, SILAGE, IRRIGATED, PART OF CROP - OPERATI...     23.000   

                                                                count_nonzero  
state_al

If we want to see the distinct values in a column, we use df.col_name.unique() (so easy!)

Now I want to get the rows from the dataframe where the county_name is NaN... I suspect these are sub-totals at
at state or some other non-county level.

In [6]:
dfcounty = df[pd.notnull(df.county_name)]
dfharv = dfcounty[dfcounty.short_desc.str.contains('ACRES HARVESTED')]
from IPython.display import display
display(dfcounty.county_name.count())
display(dfharv.county_name.count())

5502

1231

In [7]:
dfharv.short_desc.unique()

array(['CORN, GRAIN - ACRES HARVESTED', 'CORN, SILAGE - ACRES HARVESTED',
       'CORN, GRAIN, IRRIGATED - ACRES HARVESTED',
       'CORN, SILAGE, IRRIGATED - ACRES HARVESTED',
       'CORN, GRAIN, NON-IRRIGATED - ACRES HARVESTED'], dtype=object)

In [8]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [9]:
groups = dfharv[dfharv.state_alpha=='MO'].groupby('short_desc')
moharv = groups['ValueNum'].aggregate([np.sum])

groups = dfharv[dfharv.state_alpha=='TX'].groupby('short_desc')
txharv = groups['ValueNum'].aggregate([np.sum])

modat = go.Bar(x=moharv.index,y=moharv['sum'],name='Missouri')
txdat = go.Bar(x=txharv.index,y=txharv['sum'],name='Texas')

gdat = [modat, txdat]
layout = go.Layout(
    barmode='group'
)

py.iplot(gdat,filename = 'pandas-bar-chart')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~ncalm/0 or inside your plot.ly account where it is named 'pandas-bar-chart'


In [27]:
# create a DataFrameGroupBy object
gbo = dfharv.groupby(['state_alpha','short_desc'])
display(gbo)
# type(gbo)

# we can group a groupby object if using a multi-index (i.e. more than one grouping column, as above)
stategbo = gbo.groupby(level=0)
# or
stategbo = gbo.groupby('state_alpha')
display(stategbo)

# this is a series object
sgbo = gbo['ValueNum'].sum()
display(sgbo)
# type(sgbo)

# using .aggregate returns a dataframe ¯\_(ツ)_/¯
sumgbo = gbo['ValueNum'].aggregate([np.sum])
display(sumgbo)
# type(sumgbo)



# sumgbo.groupby(level=0).sum()
# harv = groups['ValueNum'].aggregate([np.sum])
# harv.filter['state_alpha=''MO''']
#harv['MO']
#modat = go.Bar(x=moharv.index,y=moharv['sum'],name='Missouri')
#txdat = go.Bar(x=txharv.index,y=txharv['sum'],name='Texas')

#gdat = [modat, txdat]
#layout = go.Layout(
#    barmode='group'
#)

#py.iplot(gdat,filename = 'pandas-bar-chart')

AttributeError: Cannot access callable attribute 'groupby' of 'DataFrameGroupBy' objects, try using the 'apply' method